In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Essential_Thrombocythemia"
cohort = "GSE174060"

# Input paths
in_trait_dir = "../../input/GEO/Essential_Thrombocythemia"
in_cohort_dir = "../../input/GEO/Essential_Thrombocythemia/GSE174060"

# Output paths
out_data_file = "../../output/preprocess/Essential_Thrombocythemia/GSE174060.csv"
out_gene_data_file = "../../output/preprocess/Essential_Thrombocythemia/gene_data/GSE174060.csv"
out_clinical_data_file = "../../output/preprocess/Essential_Thrombocythemia/clinical_data/GSE174060.csv"
json_path = "../../output/preprocess/Essential_Thrombocythemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression data from CD34+ BMMCs/PBMCs from MPN patients vs. healthy controls"
!Series_summary	"Philadelphia chromosome-negative myeloproliferative neoplasms (MPN) consist of primary myelofibrosis (PMF), polycythemia vera (PV), essential thrombocythemia (ET) and seconday myelofibrosis (SMF), comprising post-ET-MF(pET-MF) and post-PV-MF(pPV-MF)."
!Series_summary	"In this dataset, we compare the gene expression data of bone marrow or peripheral blood mononuclear cells (BMMCs/PBMCs) of CD34+ cells from MPN patients and healthy donors."
!Series_summary	""
!Series_overall_design	"36 samples (6 ET, 11 PV, 9 PMF, 4 secondary MF (SMF) patients and 6 healthy donors) were analyzed"
Sample Characteristics Dictionary:
{0: ['tissue: bone marrow', 'tissue: peripheral blood'], 1: ['genotype: JAK2V617F', 'genotype: CALRins5', 'genotype: CALRdel52', 'genotype: MPLW515L', 'genotype: triple-negative', 'genotype: N/A'], 2: ['age: 41', 'age: 53', 'age: 52', 'age:

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
from typing import Dict, Any, Optional, Callable
import json

# 1. Gene Expression Data Availability
# Based on the Series title "Gene expression data from CD34+ BMMCs/PBMCs from MPN patients vs. healthy controls"
# and the data type description, it appears this dataset contains gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait (Essential Thrombocythemia) is available in the 'diagnosis' field (key 4)
trait_row = 4

# Age is available in key 2
age_row = 2

# Gender is available in key 3
gender_row = 3

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert the trait value to binary (0 or 1).
    0 = No Essential Thrombocythemia, 1 = Has Essential Thrombocythemia
    """
    if value is None:
        return None
    
    # Extract value after the colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check if the value indicates Essential Thrombocythemia
    if value.lower() == "et":
        return 1
    elif value.lower() in ["pv", "pmf", "ppv-mf", "pet-mf", "healthy control"]:
        return 0
    return None

def convert_age(value):
    """
    Convert the age value to continuous numeric data.
    """
    if value is None:
        return None
    
    # Extract value after the colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """
    Convert gender value to binary (0 for female, 1 for male).
    """
    if value is None:
        return None
    
    # Extract value after the colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == "F":
        return 0
    elif value.upper() == "M":
        return 1
    return None

# 3. Save Metadata - Initial Filtering
# Check if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
if trait_row is not None:
    # Assuming clinical_data is a DataFrame from a previous step that contains the sample characteristics
    # We need to first load it from the cohort directory
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path)

        # Extract clinical features using the geo_select_clinical_features function
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )

        # Preview the extracted features
        preview = preview_df(clinical_features)
        print("Preview of clinical features:", preview)

        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save clinical features to CSV
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical features saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1',
       'TC01000004.hg.1', 'TC01000005.hg.1', 'TC01000006.hg.1',
       'TC01000007.hg.1', 'TC01000008.hg.1', 'TC01000009.hg.1',
       'TC01000010.hg.1', 'TC01000011.hg.1', 'TC01000012.hg.1',
       'TC01000013.hg.1', 'TC01000014.hg.1', 'TC01000015.hg.1',
       'TC01000016.hg.1', 'TC01000017.hg.1', 'TC01000018.hg.1',
       'TC01000019.hg.1', 'TC01000020.hg.1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, I can see they follow a pattern like "TC01000001.hg.1"
# These are not standard human gene symbols (like BRCA1, TP53, etc.)
# They appear to be probe IDs from a microarray platform, likely Affymetrix
# These will need to be mapped to standard human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST0000040838

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in gene_annotation correspond to gene identifiers and gene symbols
# Looking at the gene_annotation preview, we can see:
# - The 'ID' column matches the gene identifiers in gene_data.index (e.g., TC01000001.hg.1)
# - The 'gene_assignment' column contains gene symbol information

# 2. Get gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')

# Print the first few rows of the mapping to verify
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print shape information of the mapped gene data
print(f"Gene data shape after mapping: {gene_data.shape}")
print("First 5 gene symbols after mapping:")
print(gene_data.index[:5])


Gene mapping preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'Gene': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST00000408384 // MIR1302-9 // microRNA 1302-9 // --- // 100422831 /// ENST00000408384 // MIR1302-2 // microRNA 1302-2 // --- // 100302278 /// ENST00000469289 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000469289 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST00000469289 // MIR1302-9 // microRNA 1302-9 // --- // 100422831 /// ENST00000469289 // MIR1302-2 /

Gene data shape after mapping: (71527, 36)
First 5 gene symbols after mapping:
Index(['A-', 'A-2', 'A-52', 'A-575C2', 'A-E'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from patients with Essential Thrombocythemia (ET), Polycythemia Vera (PV), and Primary Myelofibrosis (PMF)."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...
Gene data shape after normalization: (24018, 36)


Normalized gene data saved to ../../output/preprocess/Essential_Thrombocythemia/gene_data/GSE174060.csv
Loading the original clinical data...
Extracting clinical features...
Clinical data preview:
{'GSM5285411': [1.0, 41.0, 0.0], 'GSM5285412': [1.0, 53.0, 0.0], 'GSM5285413': [1.0, 52.0, 1.0], 'GSM5285414': [1.0, 47.0, 1.0], 'GSM5285415': [1.0, 19.0, 1.0], 'GSM5285416': [1.0, 33.0, 0.0], 'GSM5285417': [0.0, 58.0, 1.0], 'GSM5285418': [0.0, 76.0, 1.0], 'GSM5285419': [0.0, 68.0, 1.0], 'GSM5285420': [0.0, 65.0, 0.0], 'GSM5285421': [0.0, 61.0, 1.0], 'GSM5285422': [0.0, 45.0, 1.0], 'GSM5285423': [0.0, 68.0, 0.0], 'GSM5285424': [0.0, 42.0, 1.0], 'GSM5285425': [0.0, 36.0, 0.0], 'GSM5285426': [0.0, 42.0, 1.0], 'GSM5285427': [0.0, 69.0, 0.0], 'GSM5285428': [0.0, 45.0, 1.0], 'GSM5285429': [0.0, 74.0, 1.0], 'GSM5285430': [0.0, 75.0, 1.0], 'GSM5285431': [0.0, 62.0, 1.0], 'GSM5285432': [0.0, 58.0, 1.0], 'GSM5285433': [0.0, 76.0, 1.0], 'GSM5285434': [0.0, 72.0, 0.0], 'GSM5285435': [0.0, 61.0, 0.0], 'G

Linked data shape after handling missing values: (36, 24021)
Checking for bias in trait distribution...
For the feature 'Essential_Thrombocythemia', the least common label is '1.0' with 6 occurrences. This represents 16.67% of the dataset.
The distribution of the feature 'Essential_Thrombocythemia' in this dataset is fine.

Quartiles for 'Age':
  25%: 39.75
  50% (Median): 54.5
  75%: 68.0
Min: 19.0
Max: 76.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 17 occurrences. This represents 47.22% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset usability: True


Linked data saved to ../../output/preprocess/Essential_Thrombocythemia/GSE174060.csv
